Summary things learn:

* you can do multiline to create a single line by using (). this joins the text to line one
* you can do the same with """ """" for multiline text but this one keeps the tabs and new line
* jupyter's api https://github.com/jupyter/jupyter/wiki/Jupyter-Notebook-Server-API#Sessions-API
* how to change config with command line jupter lab --help-all

In [1]:
import requests
from IPython.display import JSON

source https://stackoverflow.com/questions/41782255/how-to-get-the-current-jupyter-notebook-servers-in-python
https://jupyter-notebook.readthedocs.io/en/stable/

this doesn't work. only for jupyter server and not sagemaker
https://jupyterhub.readthedocs.io/en/stable/reference/config-user-env.html

```python
%%javascript  
IPython.notebook.save_notebook()
```

```python
!jupyter notebook list
```


To see all available configurables, use `--help-all`

Examples
--------

    jupyter notebook                       # start the notebook
    jupyter notebook --certfile=mycert.pem # use SSL/TLS certificate
    jupyter notebook password              # enter a password to protect the server

## Where the magic happens

TODO

* Need to add psutil get the average cpu/gpu if people are using terminal to run code instead of notebook
* should add a time range that is related to office hour t =o not run. 
* something to do with gpu usage as well. 
* I can totally unit test this..
* put this at github totally!. 
* add cron tutorial on readme so anyone can use it without understanding. like cron second hour day month dayof week etc...
* tutorial on how to run this...

## Obtain jupyter's status

In [4]:
# this code follows below links
# https://github.com/jupyter/notebook/blob/f354740e57f206d67bfb077a9f23bb8d22b6b311/notebook/notebookapp.py#L413
# https://github.com/aws-samples/amazon-sagemaker-notebook-instance-lifecycle-config-samples/blob/master/scripts/auto-stop-idle/autostop.py

from notebook import notebookapp
import requests
from datetime import datetime, timedelta
import json
import click

server_info = list(notebookapp.list_running_servers())[0]  # assume there are only 1 jupyter session
url = server_info['url']
headers = {'Authorization': 'token ' + server_info['token'], 
           'Connection':'close'}  # https://stackoverflow.com/a/15511852
# https://github.com/jupyter/jupyter/wiki/Jupyter-Notebook-Server-API#Sessions-API
# can also with api/kernels instead
response = requests.get(url + 'api/sessions', 
                        headers=headers, 
                        verify=False)  # https://requests.readthedocs.io/en/master/user/advanced/
data = response.json()

In [5]:
data

[{'id': '2868c06b-2ea6-4d32-9b0c-f54959f07710',
  'path': 'Untitled.ipynb',
  'name': '0f6f06aa-d305-442a-86ae-b1ab1399a0f0',
  'type': '',
  'kernel': {'id': '04f2750f-b609-49e6-aa15-345bede90119',
   'name': 'python3',
   'last_activity': '2020-01-23T06:45:53.672333Z',
   'execution_state': 'idle',
   'connections': 1}}]

---

## Command line interface

In [ ]:
import click
## TODO: Add how often you want to check?? I think that is for the cron no??

@click.command()
@click.option(
    '--port', '-p',
    default='8443',
    help='port number of the jupyter server. default is 8443'
)
@click.option(
    '--idle-time', '-t',
    default=3600,
    help='Auto stop time in seconds. default 1 hour'
)
@click.option(
    '--ignore-connections',
    is_flag=True,
    help='Stop notebook once idle, ignore connected users'
)

# Auto shutdown is stateless. Each time it runs, it check if it is idle
def main(port, idle_time, ignore_connections):
    print("Starting auto shutdown")
    print(f'args {port}, {idle_time}, {ignore_connections}')


if __name__ == "__main__":
    main()
    

---

## Determine if idle

In [ ]:
# not sure what this does...
# import urllib3
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [14]:
# [n.get('kernel', {}).get('execution_state') for n in data]

['idle', 'idle', 'busy']

In [ ]:
# def is_idle(last_activity):
#     last_activity = datetime.strptime(last_activity,"%Y-%m-%dT%H:%M:%S.%fz")
#     # jupyter use utc time
#     if (datetime.utcnow() - last_activity).total_seconds() > time:
#         return True
#     else:
#         return False

In [35]:
# Read in command-line parameters
# idle = True
# ignore_connections = False
# idle_max = 60*60  # 1 hour

In [37]:
def time_diff(last_activity):
    last_activity = datetime.strptime(last_activity,"%Y-%m-%dT%H:%M:%S.%fz")
    return (datetime.utcnow() - last_activity).total_seconds()

# TODO: not sure what this do
def get_notebook_name():
    log_path = '/opt/ml/metadata/resource-metadata.json'
    with open(log_path, 'r') as logs:
        _logs = json.load(logs)
    return _logs['ResourceName']

True

In [ ]:
def is_idle(data):
    """Determine if jupyter is idle based on inputs"""
    idle_min = idle_time
    idle = True
    for notebook in data:
        # Idleness is defined by Jupyter
        # https://github.com/jupyter/notebook/issues/4634
        kernel_info = notebook.get('kernel', {})
        # As long as there is atleast 1 active notebook
        if kernel_info.get('execution_state') == 'busy':
            idle = False
        
        # As long as someone is connected to a notebook
        if not ignore_connections & kernel_info.get('connections', 0) > 0:
            idle = False
        
        # idle in seconds based on the latest activity for all notebook
        if kernel_info.get('execution_state') == 'idle':
            idle_min = min(time_diff(noteb), idle_min)
            if idle_min >= idle_time:
                idle = False
        
        # It only takes one non-idle connection
        if not idle:
            break
        
    return idle

In [20]:
if len(data) > 0:
    idle = is_idle(data)

# Use case is when SageMaker first launch with has zero active notebooks so need to give it some time
# Also it will shutdown once all notebooks are closed. 
else:
    # simplier way could be just 
    client = boto3.client('sagemaker')
    uptime = client.describe_notebook_instance(
        NotebookInstanceName=get_notebook_name()
    )['LastModifiedTime']
    # NOT sure if sagemaker is using utc??
    # but I should be using time_diff... 
    # Bug? This is shutdown once there are zero notebook. If you just happens to close
    # everything during the time cron checks, it will shutdown..
    if not is_idle(uptime.strftime("%Y-%m-%dT%H:%M:%S.%fz")):
        idle = False

        
# TODO: Add the ability to have psutil cpu/gpu monitoring? Just in case script is using terminal instead
        # simple way is just to do monitoring of cpu/gpu for the next ten minutes if below then shut down. 
        # should record idle sttate to see how much is being used...
if idle:
    client = boto3.client('sagemaker')
    client.stop_notebook_instance(
        NotebookInstanceName=get_notebook_name()
    )

datetime.datetime(2020, 1, 20, 19, 48, 10, 446475)

In [48]:
# https://stackoverflow.com/a/2175170
UTC_OFFSET_TIMEDELTA = datetime.utcnow() - datetime.now()

## End of magic
---

## Random save

In [18]:
tmp = ("testing multiple lines"
       "wassup this is 2nd line"
       "third line reporting for duty")
tmp

'testing multiple lineswassup this is 2nd linethird line reporting for duyt'

In [19]:
tmp = ("""testing multiple lines 
        wassup this is 2nd line
       third line reporting for duty""")
tmp

'testing multiple lines \n        wassup this is 2nd line\n       third line reporting for duty'

In [ ]:
shutdown_no_activity_timeout

In [15]:
tmp = datetime.utcnow()
tmp

datetime.datetime(2020, 1, 23, 7, 10, 37, 864361)

In [27]:
tmp3 = datetime.utcnow()
tmp3

datetime.datetime(2020, 1, 23, 7, 14, 14, 534918)

In [33]:
int((tmp3 - tmp).total_seconds())

216

In [24]:
tmp2 = tmp + timedelta(seconds=10000)
tmp2

datetime.datetime(2020, 1, 23, 9, 57, 17, 864361)

In [21]:
(tmp2 - tmp) == timedelta(seconds=60)

True

In [25]:
(tmp2 - tmp).total_seconds()

10000.0

In [26]:
(tmp2 - tmp)

datetime.timedelta(seconds=10000)